In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import arch.bootstrap as boot
import importlib

import seaborn as sns
import time

from sklearn.cluster import KMeans
import sklearn

# my modules

import sys
sys.path.insert(0, r"C:\Users\johnsorensen\Documents\Python Package")


plt.rcParams.update({'font.size': 10})
plt.rcParams['figure.dpi'] = 75

%matplotlib inline

In [2]:
from optimization_new import problem, metrics, resampling, constraints
from optimization_new import visualizations as vis

importlib.reload(constraints)
importlib.reload(metrics)
importlib.reload(problem)
importlib.reload(resampling)
importlib.reload(vis)

<module 'optimization_new.visualizations' from 'C:\\Users\\johnsorensen\\Documents\\Python Package\\optimization_new\\visualizations.py'>

In [3]:
shared_folder = "F:\SITFO\Risk\Corpus and Drawdown"
saa_returns = pd.read_excel(shared_folder + r"\SAA data.xlsx", "Data", header = [0, 1])
saa_returns.index = pd.period_range(start = "1990-02", end = "2023-04", freq = "M")
saa_returns = saa_returns.drop("Portfolio", axis = 1, level = 1).dropna()
saa_returns.columns.names = ['GRID Bucket', 'Asset Class']

weights = pd.read_excel(shared_folder + r"\Weight Scenarios.xlsx", 
                               "Weights", index_col = [0, 1], nrows = 10)

policy_weights = weights["Policy Weights"]

saa_returns /= 100

In [4]:
def problem_func(returns):
    
    mean = returns.mean(axis = 0).values
    std = returns.std(axis = 0).values
    
    m1 = metrics.Volatility(returns.cov().values, annualization = "Sqrt")
    m2 = metrics.AvgArithReturn(mu = mean, annualization = "Compound")
    
    return problem.PortfolioProblem(returns, [m1, m2])
    
    
true_opt_obj = problem_func(saa_returns)
true_opt_obj.optimize()

In [6]:
#rolling_resampler = resampling.RollingWindow(problem_func, args_to_resample = [saa_returns],
#                                             gap_size = 12, ascending = False)

In [7]:
#rolling_resampler.resample()


In [ ]:
boot_resampler = resampling.